# Categorías y criterios de la Lista Roja de la UICN

Este [Jupyter Notebook](https://jupyter.org/), desarrollado en el lenguaje de programación [Python](https://www.python.org/), aplica las categorías y criterios de la [Lista Roja de la Unión Internacional para la Conservación de la Naturaleza (UICN)](https://www.iucnredlist.org/es/).

**Entradas**
- Un archivo CSV con nombres científicos de especies.

**Procesamiento**
- Se obtienen las llaves (*keys*) de las especies en el [API de GBIF](https://www.gbif.org/developer/summary).
- En caso de ser necesario, se filtran las llaves (ej. se conservan solo las de nombres que tengan coincidencia exacta o solo las de nombres aceptados).
- Con base en la lista de llaves, se construye una consulta para el [portal de GBIF](https://www.gbif.org/).
- El resultado de la consulta al portal se descarga y se renombra como "occurrences.csv".
- Se recorre el archivo "occurrences.csv" para generar las salidas que se describen seguidamente.

**Salidas**

Para cada especie en el archivo de entrada, se genera:
- Un registro en un archivo CSV con las siguientes columnas:
  - Extensión de presencia (EOO).
  - Área de ocupación (AOO).
  - Lista de países con registros de presencia.
- Un mapa de registros de presencia.
- Un mapa de registros de presencia agrupados (en *clusters*).
- Un mapa de calor.

**Bibliotecas de Python**

In [1]:
import requests
import json
import io

import csv

import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.spatial import ConvexHull

import folium
from folium import plugins

import fiona
from shapely.geometry import shape, Point

from pyproj import Proj, transform

import matplotlib.pyplot as plt
%matplotlib inline

import calendar

# El siguiente archivo debe estar en el mismo directorio que este notebook
from functions_query_from_species_list import *

**Constantes**

In [2]:
# Credenciales para el API de GBIF
GBIF_USER_NAME = "mvargas"
GBIF_PASSWORD = "grizabella"
GBIF_NOTIFICATION_ADDRESSES = "mvargas@inbio.ac.cr"
GBIF_DOWNLOAD_FORMAT = "SIMPLE_CSV"

# Proyecciones cartográficas para los cálculos de EOO y AOO
INPUT_PROJECTION = Proj(init='epsg:4326')
OUTPUT_PROJECTION = Proj(init='epsg:3857')

# Límites y dimensiones de la cuadrícula para el cálculo del AOO.
# Deben especificarse en las unidades del sistema espacial de referencia (SRS) que se utiliza.
# Por ejemplo, para el caso de Web Mercator (EPSG:3857), deben especificarse en metros.
# Límites de la cuadrícula
AOO_GRID_X_MIN = -15000000
AOO_GRID_X_MAX = -4000000
AOO_GRID_Y_MIN = -7000000
AOO_GRID_Y_MAX = 7000000
# Dimensiones de la cuadrícula
AOO_GRID_CELL_X_WIDTH = 2000
AOO_GRID_CELL_Y_WIDTH = 2000
AOO_GRID_CELL_AREA = AOO_GRID_CELL_X_WIDTH * AOO_GRID_CELL_Y_WIDTH

# Directorio de entrada
INPUT_DIR = "C:/Users/mfvargas/evaluacion-arboles-mesoamerica/"

# Archivo CSV de entrada con lista de especies a procesar
INPUT_CHECKLIST = INPUT_DIR + "Abarema_racemiflora-Axinaea_costaricensis-lista-especies.csv"
# Columna con el nombre científico de la especie
INPUT_SCINAME_COL = "Taxon Name"

# Archivo CSV con registros de presencia
OCCURRENCES_CSV = INPUT_DIR + "Abarema_racemiflora-Axinaea_costaricensis-registros-presencia.csv"

# Archivo CSV con especies del archivo de entrada que no se procesan
INPUT_CHECKLIST_NON_PROCESSED = INPUT_DIR + "Abarema_racemiflora-Axinaea_costaricensis-lista-especies-no-procesadas.csv"

# Directorio de salida
OUTPUT_DIR = "C:/Users/mfvargas/evaluacion-arboles-mesoamerica/Abarema_racemiflora-Axinaea_costaricensis/"
# Archivo CSV de salida
OUTPUT_CSV = "C:/Users/mfvargas/evaluacion-arboles-mesoamerica/Abarema_racemiflora-Axinaea_costaricensis-evaluacion.csv"
# Archivo HTML de salida
OUTPUT_HTML = "C:/Users/mfvargas/evaluacion-arboles-mesoamerica/Abarema_racemiflora-Axinaea_costaricensis-evaluacion.html"
# URL de salida
OUTPUT_BASE_URL = "https://evaluacion-arboles-mesoamerica.github.io/Abarema_racemiflora-Axinaea_costaricensis/"

# Capa geoespacial de áreas protegidas
PROTECTED_AREAS_LAYER = "C:/Users/mfvargas/geodatos/wdpa/WDPA_Mesoamerica.shp"

# Número máximo de registros a desplegar en los dataframes de Pandas
pd.options.display.max_rows = 70

## Carga de datos

In [3]:
# Carga del archivo CSV de entrada en un dataframe de Pandas
input_species_df = pd.read_csv(INPUT_CHECKLIST, encoding='utf_8')

input_species_df

,Family,Genus,Species,Taxon Name,Category,Criteria,IUCN Assessment completed,Notes,Endemic?,Information inputted into SIS,Assessment reviewed,Map completed,Submitted to IUCN
0,FABACEAE,Abarema,racemiflora,Abarema racemiflora,LC,NaN,NaN,To send to David Neill for review,no,yes,NaN,yes,NaN
1,MALVACEAE,Abutilon,purpusii,Abutilon purpusii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EUPHORBIACEAE,Acalypha,ferdinandi,Acalypha ferdinandi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LAMIACEAE,Aegiphila,panamensis,Aegiphila panamensis,NaN,NaN,NaN,1998 assessment,NaN,NaN,NaN,NaN,NaN
4,OPILIACEAE,Agonandra,macrocarpa,Agonandra macrocarpa,NaN,NaN,NaN,1998 assessment,NaN,NaN,NaN,NaN,NaN
5,LAURACEAE,Aiouea,brenesii,Aiouea brenesii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LAURACEAE,Aiouea,chavarrianum,Aiouea chavarrianum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LAURACEAE,Aiouea,neurophylla,Aiouea neurophylla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LAURACEAE,Aiouea,obscura,Aiouea obscura,NaN,NaN,NaN,1998 assessment,NaN,NaN,NaN,NaN,NaN
9,LAURACEAE,Aiouea,pittieri,Aiouea pittieri,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se consulta el API de GBIF para obtener las llaves (*keys*) de las especies

In [4]:
# Se obtienen las llaves de las especies a través del API de GBIF
gbif_species_df = match_species(input_species_df, INPUT_SCINAME_COL)

gbif_species_df[['inputName', 'species', 'genus', 'family', 'matchType', 'status', 'synonym', 'speciesKey', 'usageKey', 'rank', 'alternatives']]

,inputName,species,genus,family,matchType,status,synonym,speciesKey,usageKey,rank,alternatives
0,Abarema racemiflora,Abarema racemiflora,Abarema,Fabaceae,EXACT,ACCEPTED,False,2977909,2977909,SPECIES,NULL
1,Abutilon purpusii,Callianthe purpusii,Callianthe,Malvaceae,EXACT,SYNONYM,True,8418086,3936341,SPECIES,NULL
2,Acalypha ferdinandi,Acalypha ferdinandi,Acalypha,Euphorbiaceae,EXACT,ACCEPTED,False,7276169,7276169,SPECIES,NULL
3,Aegiphila panamensis,Aegiphila panamensis,Aegiphila,Lamiaceae,EXACT,ACCEPTED,False,3891222,3891222,SPECIES,"[{'usageKey': 3891182, 'scientificName': 'Aegi..."
4,Agonandra macrocarpa,Agonandra macrocarpa,Agonandra,Opiliaceae,EXACT,ACCEPTED,False,3658839,3658839,SPECIES,"[{'usageKey': 7924587, 'scientificName': 'Adin..."
5,Aiouea brenesii,Aiouea brenesii,Aiouea,Lauraceae,EXACT,ACCEPTED,False,9665460,9665460,SPECIES,NULL
6,Aiouea chavarrianum,Aiouea chavarriana,Aiouea,Lauraceae,FUZZY,ACCEPTED,False,9695833,9695833,SPECIES,NULL
7,Aiouea neurophylla,Aiouea neurophylla,Aiouea,Lauraceae,EXACT,ACCEPTED,False,9810051,9810051,SPECIES,NULL
8,Aiouea obscura,Aiouea obscura,Aiouea,Lauraceae,EXACT,ACCEPTED,False,4183412,4183412,SPECIES,"[{'usageKey': 2358012, 'acceptedUsageKey': 520..."
9,Aiouea pittieri,Aiouea pittieri,Aiouea,Lauraceae,EXACT,ACCEPTED,False,9700381,9700381,SPECIES,NULL


In [8]:
# Nombres excluídos por no ser aceptados o con coincidencia no exacta
gbif_species_df = gbif_species_df[['inputName', 'species', 'genus', 'family', 'matchType', 'status', 'synonym', 'speciesKey', 'usageKey', 'rank', 'alternatives']]
gbif_species_non_processed_df = gbif_species_df.loc[~((gbif_species_df["matchType"]=="EXACT") & (gbif_species_df["status"]=="ACCEPTED"))]

gbif_species_non_processed_df.to_csv(INPUT_CHECKLIST_NON_PROCESSED)

In [ ]:
# Se separa la lista de llaves

# Se filtran las llaves (en caso de ser necesario)
key_list = gbif_species_df.loc[(gbif_species_df["matchType"]=="EXACT") & (gbif_species_df["status"]=="ACCEPTED")].usageKey.tolist()
# key_list = gbif_taxa_df.usageKey.tolist()

key_list

Se construye una consulta para descarga en el portal de GBIF

In [ ]:
# Se construye una consulta para descarga en el portal de GBIF
download_query = {}
download_query["creator"] = GBIF_USER_NAME
download_query["notificationAddresses"] = [GBIF_NOTIFICATION_ADDRESSES]
download_query["sendNotification"] = True
download_query["format"] = GBIF_DOWNLOAD_FORMAT
download_query["predicate"] =   {"type":"and", "predicates": 
                                 [
                                    {"type":"equals", "key":"HAS_COORDINATE",       "value":"true"},
                                    {"type":"equals", "key":"HAS_GEOSPATIAL_ISSUE", "value":"false"}, 
                                    {"type":"in",     "key": "TAXON_KEY",           "values":key_list}
                                 ]
                                }

download_query

In [ ]:
# Submit query to GBIF API
create_download_given_query(GBIF_USER_NAME, GBIF_PASSWORD, download_query)

# Respuesta esperada:
# ok
# <Response [201]>

**After downloading the file from the GBIF portal if has to be unzipped and renamed with the name defined in OCCURRENCES_CSV**

In [ ]:
occurrences_df = pd.read_csv(OCCURRENCES_CSV, sep='\t')

occurrences_df

In [ ]:
# Change "eventDate" data type to dateTime
occurrences_df["eventDate"] = pd.to_datetime(occurrences_df["eventDate"])

In [ ]:
# ==================== CREACIÓN DEL DATAFRAME DE ÁREAS PROTEGIDAS ====================
wdpa_gdf = gpd.read_file(PROTECTED_AREAS_LAYER, encoding="latin-1")
wdpa_gdf.index.name = "index_wdpa"


# ==================== CREACIÓN DE LOS ARCHIVOS DE SALIDA ====================

# Archivo CSV
results_csv = open(OUTPUT_CSV, mode='w', newline='', encoding="latin-1")
results_csv_writer = csv.writer(results_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
results_csv_writer.writerow(['Nombre cientifico de entrada', 
                             'Familia (GBIF)', 
                             'Genero (GBIF)', 
                             'Especie (GBIF)', 
                             'EOO (km2)', 
                             'AOO (km2)',
                             'Altitud minima (m)', 
                             'Altitud maxima (m)',
                             'Países', 
                             'Areas protegidas',
                             'Mapa de registros de presencia', 
                             'Mapa agrupado'])

# Archivo HTML
results_html = open(OUTPUT_HTML, mode='w', newline='', encoding="latin-1")
results_html.write("<!DOCTYPE html>")
results_html.write('<html lang="es">')
results_html.write("<head>")
results_html.write("<title>Evaluacion de arboles de Mesoamerica</title>")
results_html.write("<style>table, th, td {border: 1px solid black;}</style>")
results_html.write("</head>")
results_html.write("<body>")
results_html.write("<table>")
results_html.write("<tr><th>Nombre cientifico de entrada</th><th>Familia (GBIF)</th><th>Genero (GBIF)</th><th>Especie (GBIF)</th><th>EOO (km2)</th><th>AOO (km2)</th><th>Altitud minima (m)</th><th>Altitud maxima (m)</th><th>Paises</th><th>Areas protegidas</th><th>Mapa de registros de presencia</th><th>Mapa agrupado</th></tr>")
results_html.write("<tbody>")


# ==================== RECORRIDO DE LA LISTA DE ESPECIES ====================

for index, row in gbif_species_df.iterrows():
    species_input = row["inputName"]
    family_gbif = row["family"]
    genus_gbif = row["genus"]
    species_gbif = row["species"]
    print(species_gbif)

    current_taxon_df = occurrences_df[occurrences_df['species'] == species_gbif]    

    
    # ==================== ESTRUCTURAS PARA EL CÁLCULO DEL EOO Y DEL AOO ====================    

    # Lista de puntos para el cálculo del EOO
    eoo_points = []   
    
    # Lista de valores de (x,y) para el cálculo del AOO
    aoo_x_values = []
    aoo_y_values = []   
    
    
    # ==================== ESTRUCTURAS PARA EL CÁLCULO DE LAS ALTITUDES MÍNIMA Y MÁXIMA ====================    

    altitude_values = []

    
    # ==================== ESTRUCTURAS PARA LA GENERACIÓN DE LA LISTA DE PAÍSES ====================    
    
    # Conjunto de códigos de países en los que hay registros de presencia
    countries = set()
    

    # ==================== ESTRUCTURAS PARA LA GENERACIÓN DE LA LISTA DE ÁREAS PROTEGIDAS ====================    
    
    # Arreglo de códigos de países en los que hay registros de presencia    
    current_taxon_geom = [Point(xy) for xy in zip(current_taxon_df["decimalLongitude"], 
                                                  current_taxon_df["decimalLatitude"])]
    current_taxon_gdf = gpd.GeoDataFrame(current_taxon_df, 
                                         crs={"init": "epsg:4326"}, 
                                         geometry=current_taxon_geom)
    wdpa_ocuppied = gpd.sjoin(wdpa_gdf, current_taxon_gdf, how="inner", op='intersects')
    protected_areas = wdpa_ocuppied.NAME.unique()    
           
        
    # ==================== INICIALIZACIÓN DE MAPAS ====================
    
    # Mapa de registros de presencia
    occurrences_map = folium.Map(location=[9.63, -84], 
                                 tiles='OpenStreetMap', 
                                 attr='OpenStreetMap', 
                                 zoom_start=5, 
                                 control_scale=True)
    folium.TileLayer(tiles='http://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/MapServer/tile/{z}/{y}/{x}',
                     name='ESRI World Imagery',
                     attr='ESRI World Imagery').add_to(occurrences_map)
    
    # Mapa de registros de presencia agrupados (cluster)
    cluster_map = folium.Map(location=[9.63, -84], 
                             tiles='OpenStreetMap', 
                             attr='OpenStreetMap', 
                             zoom_start=5, 
                             control_scale=True)
    folium.TileLayer(tiles='http://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/MapServer/tile/{z}/{y}/{x}',
                     name='ESRI World Imagery',
                     attr='ESRI World Imagery').add_to(cluster_map)    
    occurrences_cluster = plugins.MarkerCluster().add_to(cluster_map)
        
    
    # ==================== RECORRIDO DE LOS REGISTROS DE PRESENCIA ====================
    for lat, lng, alt, country, m, label in zip(current_taxon_df.decimalLatitude,
                               current_taxon_df.decimalLongitude,
                               current_taxon_df.elevation,
                               current_taxon_df.countryCode,
                               current_taxon_df.eventDate.dt.month,
                               "<strong>Localidad:</strong> "       + current_taxon_df.locality.astype(str)   + "\n"    +
                               "<strong>Elevación:</strong> "       + current_taxon_df.elevation.astype(str)  + " m \n" +
                               "<strong>Fecha:</strong> "           + current_taxon_df.eventDate.astype(str)  + "\n"    +
                               "<strong>Recolectores:</strong> "    + current_taxon_df.recordedBy.astype(str) + "\n"    +
                               "<strong>Identificadores:</strong> " + current_taxon_df.identifiedBy.astype(str)):

        # Adición de valores para el cálculo de las altitudes mínima y máxima
        altitude_values.append(alt)
        
        # Adición de puntos para el cálculo del EOO
        x,y = transform(INPUT_PROJECTION, OUTPUT_PROJECTION, lng, lat)
        eoo_point = []
        eoo_point.append(x)
        eoo_point.append(y)
        eoo_points.append(eoo_point)        
        
        # Adición de valores x,y para el cálculo del AOO
        x,y = transform(INPUT_PROJECTION, OUTPUT_PROJECTION, lng, lat)
        aoo_x_values.append(x)
        aoo_y_values.append(y)
        
        # Adición de códigos de países
        countries.add(country)      
        
        # Adición de registros de presencia al mapa de registros de presencia
        folium.CircleMarker(location=[lat, lng], 
                            radius=3, 
                            color='red', 
                            fill=True,
                            popup=label,
                            fill_color='darkred',
                            fill_opacity=0.6).add_to(occurrences_map)
        
        # Adición de registros de presencia agrupados al mapa de registros de presencia agrupados (cluster)
        folium.Marker(
            location=[lat, lng],
            icon=None,
            popup=label,
        ).add_to(occurrences_cluster)
        
    
    # Cálculo de las altitudes mínima y máxima
    altitude_min = min(altitude_values)
    if (np.isnan(altitude_min)):
        altitude_min_str = ""
    else:
        altitude_min_str = "{:.2f}".format(altitude_min)
    altitude_max = max(altitude_values)
    if (np.isnan(altitude_max)):
        altitude_max_str = ""
    else:
        altitude_max_str = "{:.2f}".format(altitude_max)
    
    print("Altitud mínima:", altitude_min_str)
    print("Altitud máxima:", altitude_max_str)
    
    # Cálculo del EOO
    a = np.array(eoo_points)
    hull = ConvexHull(a)
    eoo = hull.volume / 1000000
    print("Área de extensión de presencia (EOO):", "{:.2f}".format(eoo), "km2")
    #plt.plot(a[:,0], a[:,1], 'o')
    #for simplex in hull.simplices:
    #    plt.plot(a[simplex, 0], a[simplex, 1], 'k-')  
    #plt.savefig(OUTPUT_DIR + family_gbif.upper() + "_" + species_input.replace(" ", "_") + "-grafico_eoo.png", bbox_inches='tight')    
  
    # Cálculo del AOO
    x = np.array(aoo_x_values)
    y = np.array(aoo_y_values)
    gridx = np.arange(AOO_GRID_X_MIN, AOO_GRID_X_MAX, AOO_GRID_CELL_X_WIDTH)
    gridy = np.arange(AOO_GRID_Y_MIN, AOO_GRID_Y_MAX, AOO_GRID_CELL_Y_WIDTH)
    grid, _, _ = np.histogram2d(x, y, bins=[gridx, gridy])
    occupied_cells = (grid > 0) 
    aoo = len(grid[occupied_cells]) * (AOO_GRID_CELL_AREA / 1000000)
    print("Área de ocupación (AOO):", "{:.2f}".format(aoo), "km2")
    #plt.figure()
    #plt.plot(x, y, 'ro')
    #plt.grid(True)
    ## plt.show()    
    #plt.savefig(OUTPUT_DIR + family_gbif.upper() + "_" + species_input.replace(" ", "_") + "-grafico_aoo.png", bbox_inches='tight')     
    
    # Lista de países con registros de presencia
    countries = sorted(countries)
    print("Países con registros de presencia:", countries)
    
    # Lista de áreas protegidas con registros de presencia
    protected_areas = sorted(protected_areas)
    print("Áreas protegidas con registros de presencia:", protected_areas)      
        
    # Adición de controles de capas
    folium.LayerControl().add_to(occurrences_map)
    folium.LayerControl().add_to(cluster_map)
   
    
    # Grabado de archivos HTML con los mapas
    occurrences_map.save(OUTPUT_DIR + family_gbif.upper() + "_" + species_input.replace(" ", "_") + "-mapa_registros_presencia.html")
    cluster_map.save(OUTPUT_DIR + family_gbif.upper() + "_" + species_input.replace(" ", "_") + "-mapa_agrupado.html")  
    
    # Adición de línea en el archivo CSV de salida
    results_csv_writer.writerow([species_input, 
                                 family_gbif, 
                                 genus_gbif, 
                                 species_gbif, 
                                 "{:.2f}".format(eoo), 
                                 "{:.2f}".format(aoo),
                                 altitude_min_str, 
                                 altitude_max_str,                                  
                                 ', '.join(countries),
                                 ', '.join(protected_areas),
                                 '=HYPERLINK("'+OUTPUT_BASE_URL + family_gbif.upper() + "_" + species_input.replace(" ", "_") + '-mapa_registros_presencia.html' + '", "' + 'Enlace al mapa")',
                                 '=HYPERLINK("'+OUTPUT_BASE_URL + family_gbif.upper() + "_" + species_input.replace(" ", "_") + '-mapa_agrupado.html'            + '", "' + 'Enlace al mapa")'])
    
    # Adición de línea en el archivo HTML de salida
    results_html.write("<tr>")
    results_html.write("<td>"+ species_input                 +"</td>")
    results_html.write("<td>"+ family_gbif                   +"</td>")
    results_html.write("<td>"+ genus_gbif                    +"</td>")
    results_html.write("<td>"+ species_gbif                  +"</td>")
    results_html.write("<td>"+ "{:.2f}".format(eoo)          +"</td>")
    results_html.write("<td>"+ "{:.2f}".format(aoo)          +"</td>")
    results_html.write("<td>"+ altitude_min_str              +"</td>")
    results_html.write("<td>"+ altitude_max_str              +"</td>")    
    results_html.write("<td>"+ ', '.join(countries)          +"</td>")
    results_html.write("<td>"+ ', '.join(protected_areas)    +"</td>")
    results_html.write("<td>"+ '<a href="' + OUTPUT_BASE_URL + family_gbif.upper() + '_' + species_input.replace(' ', '_') + '-mapa_registros_presencia.html' +'">Enlace</a>' + "</td>")
    results_html.write("<td>"+ '<a href="' + OUTPUT_BASE_URL + family_gbif.upper() + '_' + species_input.replace(' ', '_') + '-mapa_agrupado.html'            +'">Enlace</a>' + "</td>")
    results_html.write("</tr>")
    
    # Archivo de registros de presencia de la especie en archivo CSV
    current_taxon_df = current_taxon_df[['basisOfRecord', 'species', 'catalogNumber', 'recordNumber', 'decimalLatitude', 'decimalLongitude', 'locality', 'year', 'recordedBy']]
    current_taxon_df.columns = ['BasisOfRec', 'Binomial', 'CatalogNo', 'CollectID', 'Dec_Lat', 'Dec_Long', 'Dist_comm', 'Event_Year', 'recordedBy']
    current_taxon_df.insert(3,  "Citation",  "GBIF")
    current_taxon_df.insert(5,  "Compiler",  "Manuel Vargas")
    current_taxon_df.insert(6,  "Data_sens", "")    
    current_taxon_df.insert(11, "Origin", 1)    
    current_taxon_df.insert(12, "Presence", 1)
    current_taxon_df.insert(13, "Seasonal", 1)        
    current_taxon_df.insert(14, "Sens_comm", "")
    current_taxon_df.insert(15, "SpatialRef", "WGS84")
    current_taxon_df.insert(16, "YrCompiled", 2019)    
    current_taxon_df.to_csv(OUTPUT_DIR + family_gbif.upper() + "_" + species_input.replace(" ", "_") + "-registros_presencia.csv")
    
# ==================== CIERRE DE LOS ARCHIVOS DE SALIDA ====================

# Archivo CSV
results_csv.close()

# Archivo HTML
results_html.write("</tbody></table></body></html>")
results_html.close()